# Tech Jam 2019

##### Jakkrit sonsanit , Wacharit sirigate
***

### Import Library

In [221]:
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

### Set Random Seed

In [222]:
random.seed(1238471)

### Import demographics 

In [223]:
demo = pd.read_csv('data_pack/demographics.csv')
demo.head(10)

,id,cc_no,gender,ocp_cd,age
0,1,1,2,9.0,5
1,1,98397,2,9.0,5
2,2,2,2,3.0,4
3,2,9740,2,3.0,4
4,3,3,2,1.0,3
5,4,4,2,3.0,5
6,4,86813,2,3.0,5
7,5,5,2,9.0,4
8,6,6,1,3.0,3
9,6,91379,1,3.0,3


### Random fill ocp_cd by random

In [224]:
demo = demo.fillna(value={'ocp_cd':random.choice([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0])})

### Import kplus

In [225]:
kplus = pd.read_csv('data_pack/kplus.csv')
kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
0,14802,2018-01-14,2,2400
1,14802,2018-04-01,9,33900
2,14802,2018-05-27,7,6100
3,14802,2018-01-28,3,8500
4,14802,2018-03-25,2,4000


### Set type value in column sunday to datetime and drop column count

In [226]:
kplus.sunday = pd.to_datetime(kplus.sunday)
kplus = kplus.drop(columns=['kp_txn_count'])

### Group by id 

In [227]:
kp_month = kplus.groupby([kplus['id'], kplus['sunday'].dt.month]).sum().astype(int).reset_index()

In [228]:
kp_month.head(10)

,id,sunday,kp_txn_amt
0,1,6,3500
1,2,1,7400
2,2,2,19600
3,2,3,23500
4,2,4,63400
5,2,5,24900
6,2,6,71200
7,3,1,15800
8,3,2,14700
9,3,3,20200


### import cc groupby month from indy and drop column post_dt

In [229]:
cc_month = pd.read_csv('data_pack/sum_monthly_cc_txn_amt.csv')
cc_month.head(10)

,cc_no,pos_dt,cc_txn_amt
0,2,3,4600
1,2,4,18700
2,2,5,1600
3,2,6,2000
4,4,5,300000
5,4,6,240000
6,5,1,7000
7,5,2,5000
8,6,6,32000
9,7,1,900


### Duplicate demographic and set index by cc_no

In [230]:
demo_cc = demo.sort_values('cc_no')
demo_cc = demo_cc.set_index('cc_no')

### Map cc group by month to new demographic

In [231]:
for i in range(1,7):
    tmp = cc_month[cc_month['pos_dt'] == i]
    tmp = tmp.set_index('cc_no')
    demo_cc['cc_month_'+str(i)] = tmp.cc_txn_amt

In [232]:
demo_f = demo_cc.groupby('id').mean()
demo_f.sample(10)

,gender,ocp_cd,age,cc_month_1,cc_month_2,cc_month_3,cc_month_4,cc_month_5,cc_month_6
id,,,,,,,,,
64584,2,1.0,4,13000.0,3700.0,10400.0,8700.0,3600.0,20500.0
11622,1,3.0,6,NaN,NaN,23000.0,2900.0,1200.0,4900.0
41360,1,3.0,5,NaN,NaN,NaN,19500.0,NaN,NaN
20795,2,3.0,4,NaN,NaN,2300.0,4600.0,25700.0,NaN
48534,1,9.0,4,26800.0,18500.0,54100.0,24400.0,33300.0,18000.0
50915,1,3.0,5,NaN,NaN,NaN,NaN,NaN,48400.0
45439,1,9.0,6,8000.0,8000.0,8000.0,4000.0,8500.0,9000.0
45362,2,9.0,5,NaN,NaN,NaN,NaN,NaN,NaN
29185,1,3.0,4,3100.0,33100.0,14600.0,28400.0,25450.0,10900.0


In [233]:
for i in range(1,7):
    tmp = kp_month[kp_month['sunday'] == i]
    tmp = tmp.set_index('id')
    demo_f['kp_month_'+str(i)] = tmp.kp_txn_amt

In [234]:
demo_f = demo_f.fillna(0)
demo_f.head(10)

,gender,ocp_cd,age,cc_month_1,cc_month_2,cc_month_3,cc_month_4,cc_month_5,cc_month_6,kp_month_1,kp_month_2,kp_month_3,kp_month_4,kp_month_5,kp_month_6
id,,,,,,,,,,,,,,,
1,2,9.0,5,4700.0,20000.0,0.0,0.0,10000.0,10000.0,0.0,0.0,0.0,0.0,0.0,3500.0
2,2,3.0,4,6600.0,5700.0,4350.0,15200.0,1600.0,6100.0,7400.0,19600.0,23500.0,63400.0,24900.0,71200.0
3,2,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
4,2,3.0,5,0.0,0.0,0.0,230000.0,300000.0,240000.0,1056700.0,859000.0,715300.0,1305500.0,729000.0,1410600.0
5,2,9.0,4,7000.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,3.0,3,0.0,0.0,0.0,0.0,0.0,32000.0,0.0,0.0,0.0,0.0,0.0,13400.0
7,1,3.0,5,1150.0,2800.0,2000.0,1500.0,1850.0,11250.0,30700.0,111000.0,106400.0,93300.0,107000.0,155400.0
8,2,9.0,3,4900.0,2000.0,2200.0,2000.0,2800.0,2400.0,133600.0,133000.0,155900.0,131000.0,137700.0,113600.0
9,1,9.0,4,2000.0,10000.0,2000.0,4000.0,3000.0,5000.0,0.0,0.0,800.0,0.0,3300.0,500.0


In [235]:
demo_train = demo_f.loc[1:50000]
demo_test = demo_f.loc[50001:]

In [236]:
train = pd.read_csv('data_pack/train.csv')

In [237]:
reg = LinearRegression().fit(demo_train, train.income)
reg.score(demo_train, train.income)

0.11539795943217868

In [271]:
test = pd.read_csv('data_pack/test.csv')
test['income'] = reg.predict(demo_test)
# test.to_csv('test_v3.csv', index=False)

In [273]:
ls = test.income 
lower, upper = 0.1, 0.5
l_norm = [lower + (upper - lower) * i for i in ls]
test['income'] = l_norm

In [277]:
test.to_csv('test_v4.csv', index=False)